*langchain*

In [1]:
!pip install requests-cache python-dotenv 


In [2]:
import requests
import requests_cache
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:

# Cargar las variables de entorno
load_dotenv()
api_key = os.getenv('NEWS_API_KEY')

# Configurar el caché con una expiración de 1 día
requests_cache.install_cache('news_cache', expire_after=timedelta(days=1))

# Lista de empresas
empresas = [

    "Psyence Biomed",
    "Silo Pharma",
    "optimi health",
    "Tilray",
    "Numinus Wellness",
    "MindMed",
    "Compass Pathways",
    "Field Trip Health",
    "Cybin",
    "ATAI Life Sciences",
    "Seelos Therapeutics",
    "Mind Cure Health",
    "Tryp Therapeutics",
    "Revive Therapeutics",
    "Red Light Holland",
    "Hollister Biosciences",
    "Mydecine Innovations Group",
    "Numinus Wellness",
    "cronosgroup",
    "Ascend Wellness Holdings",
    "Verano Holdings",
    "Columbia Care",
    "Green Thumb Industries",
    "Curaleaf Holdings",
    "Trulieve Cannabis",
    "SNDL Inc",
    "IM Cannabis",
    "4front ventures",
    "Ayr Wellness",
    "TerrAscend",
    "Cresco Labs",
    "Harvest Health & Recreation",
    "Jushi Holdings",
    "MedMen",
    "Planet 13 Holdings",
    "Vireo Health",
    "Cannabis One",
    "Cansortium",
    "Columbia Care",
    "AFC Gamma",
       "Cannabis legalization",
    "Medical marijuana",
    "Cannabis industry",
    "Cannabis investment",
    "Cannabis stocks",
    "Cannabis research",
    "Cannabis news",
    "Cannabis policy",
    "Cannabis business",
    "Cannabis culture",
    "Psilocybin"
    "Psychedelics research",
    "Psychedelics industry",
    "Psychedelics investment",
    "Psychedelics news",
    "Psychedelics policy",
    "Psychedelics business",
    "Psychedelics culture"  
]

url = 'https://newsapi.org/v2/everything'

# Definir la fecha de inicio (últimos 1 día)
from_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')

news_data = []


for empresa in empresas:
    print(f"Buscando noticias para: {empresa}")
    # Ajustar la consulta para incluir términos específicos
    query = f'"{empresa}" AND (finance OR market OR legal OR investment)'
    params = {
        'q': query,
        'apiKey': api_key,
        'sortBy': 'publishedAt',  # Ordenar por fecha de publicación
        'language': 'en',         # Idioma de las noticias
        'from': from_date         # Fecha desde la cual buscar noticias
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        articles = response.json().get('articles', [])
        for article in articles:
            news_item = {
                'empresa': empresa,
                'title': article['title'],
                'link': article['url'],
                'snippet': article['description'],
                'published_at': article['publishedAt']
            }
            news_data.append(news_item)
        print(f"Encontradas {len(articles)} noticias para {empresa}")
    except requests.exceptions.RequestException as e:
        print(f"Error al buscar noticias para {empresa}: {str(e)}")

# No guardar en archivo JSON, continuar con el procesamiento en memoria

Buscando noticias para: Psyence Biomed
Encontradas 3 noticias para Psyence Biomed
Buscando noticias para: Silo Pharma
Encontradas 0 noticias para Silo Pharma
Buscando noticias para: optimi health
Encontradas 0 noticias para optimi health
Buscando noticias para: Tilray
Encontradas 6 noticias para Tilray
Buscando noticias para: Numinus Wellness
Encontradas 0 noticias para Numinus Wellness
Buscando noticias para: MindMed
Encontradas 1 noticias para MindMed
Buscando noticias para: Compass Pathways
Encontradas 0 noticias para Compass Pathways
Buscando noticias para: Field Trip Health
Encontradas 0 noticias para Field Trip Health
Buscando noticias para: Cybin
Encontradas 0 noticias para Cybin
Buscando noticias para: ATAI Life Sciences
Encontradas 0 noticias para ATAI Life Sciences
Buscando noticias para: Seelos Therapeutics
Encontradas 2 noticias para Seelos Therapeutics
Buscando noticias para: Mind Cure Health
Encontradas 0 noticias para Mind Cure Health
Buscando noticias para: Tryp Therape

In [4]:
!pip install --upgrade langchain chromadb langchain-text-splitters openai tiktoken


In [5]:
!pip install -U langchain-community

In [6]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pprint

# Cargar las variables de entorno
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')


# Procesar los datos en memoria
extracted_elements = []

for article in news_data:
    title = article.get('title', 'No title')
    snippet = article.get('snippet', 'No snippet')
    link = article.get('link', 'No link')
    content = f"Title: {title}\nSnippet: {snippet}\nLink: {link}"
    extracted_elements.append(content)


# Filtrar empresas sin noticias
filtered_elements = [element for element in extracted_elements if 'Title: No title'or'snippet:Nosi' not in element]

# Crear documentos para Langchain
documents = [Document(page_content=element) for element in filtered_elements]


# Dividir los documentos en trozos más pequeños
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# Crear embeddings y base de datos Chroma
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
db = Chroma.from_documents(docs, embeddings)

# Inicializar el modelo y el retriever
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7, openai_api_key=openai_key)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

/tmp/ipykernel_29057/965106478.py:39: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
/tmp/ipykernel_29057/965106478.py:43: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7, openai_api_key=openai_key)


In [7]:
# Generar resúmenes de las noticias
summaries = []
for doc in docs:
    query = f"Por favor, resume la siguiente noticia:\n\n{doc.page_content}: y muestrame el enlace"
    summary = qa_chain.run(query)
    summaries.append(summary)




/tmp/ipykernel_29057/1300217502.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = qa_chain.run(query)


In [8]:
# Imprimir los resúmenes individuales
for i, summary in enumerate(summaries):
    print(f"Resumen {i+1}:\n{summary}\n")

Resumen 1:
Psyence Biomed ha recibido una actualización sobre su desarrollo de ingredientes farmacéuticos activos (API) certificados por las Buenas Prácticas de Manufactura (GMP) en la Unión Europea, destacando que PsyLabs proporcionará exclusivamente psilocibina de alta pureza y grado farmacéutico para el tratamiento del trastorno por consumo de alcohol. 

Puedes leer más en el siguiente enlace: [Psyence Biomed Provides Progress Update on PsyLabs E.U. Good Manufacturing Practices (GMP) Certified Active Pharmaceutical Ingredient (API) Development](https://www.globenewswire.com/news-release/2024/10/17/2964738/0/en/Psyence-Biomed-Provides-Progress-Update-on-PsyLabs-E.U-Good-Manufacturing-Practices-GMP-Certified-Active-Pharmaceutical-Ingredient-API-Development.html)

Resumen 2:
Psyence Group Inc ha anunciado que ha celebrado un acuerdo de intercambio de deuda por acciones con su asociado listado en NASDAQ, Psyence Biomedical. Este acuerdo se denomina "Debt Swap Agreement". 

Puedes leer m